## Glue more beer checkins onto the big dataframe

In [1]:
import pandas as pd


Separate the 'description' column  
##### ========================================================================================

In [3]:
bdf = pd.read_csv('bigBeerFrame.csv')
bdf.head()

,checkin_id,beer_id,user_id,rating_user,brewery_name,beer_name,beer_style,brewery_id,brewery_type,brewery_country,...,venue_country,venue_state,venue_cat,venue_id,checkin_comment,venue_type,rating_global,description,abv,date
0,843971771,526725,1373368,3.75,Wicks Brewing,Battle Cry IPA,IPA - American,44236,Brew Pub,United States,...,United States,CA,[],3966054.0,NaN,Nightlife Spot,3.63385,NaN,6.8,2019-12-21 05:52:07+00:00
1,843407973,526725,3484155,3.50,Wicks Brewing,Battle Cry IPA,IPA - American,44236,Brew Pub,United States,...,United States,CA,[],9586831.0,Brewed for the House.,Arts & Entertainment,3.63385,NaN,6.8,2019-12-20 04:26:42+00:00
2,843380508,526725,342359,4.00,Wicks Brewing,Battle Cry IPA,IPA - American,44236,Brew Pub,United States,...,United States,CA,[],3966054.0,NaN,Nightlife Spot,3.63385,NaN,6.8,2019-12-20 02:52:06+00:00
3,837180407,526725,6700685,4.50,Wicks Brewing,Battle Cry IPA,IPA - American,44236,Brew Pub,United States,...,NaN,NaN,[],NaN,NaN,NaN,3.63385,NaN,6.8,2019-12-03 21:53:09+00:00
4,836736462,526725,3094250,3.50,Wicks Brewing,Battle Cry IPA,IPA - American,44236,Brew Pub,United States,...,NaN,NaN,[],NaN,NaN,NaN,3.63385,NaN,6.8,2019-12-01 21:53:24+00:00


In [4]:
# make a new frame just for the descriptions
description = pd.DataFrame({'beer_id': bdf.beer_id, 'beer_description': bdf.description})

In [5]:
bigframe_descrips = pd.read_csv('descriptions.csv')
bigframe_descrips.shape

(1425447, 3)

In [6]:
bigframe_descrips.columns

Index(['Unnamed: 0', 'beer_id', 'beer_description'], dtype='object')

In [8]:
bigframe_descrips = bigframe_descrips[bigframe_descrips.columns[1:]]

In [24]:
descrips = pd.concat([bigframe_descrips, description])

In [25]:
descrips.shape

(2061965, 2)

In [26]:
# remove NaN's first and then remove duplicates.
descrips.dropna(inplace=True)
descrips.shape

(1504358, 2)

In [27]:
descrips = descrips.drop_duplicates(subset=['beer_id','beer_description'])

In [28]:
descrips.shape

(114348, 2)

In [38]:
descrips.groupby('beer_id').size().sort_values(ascending=False)[:5]

beer_id
3366861    2
3540435    1
2019667    1
2019896    1
2019929    1
dtype: int64

In [37]:
descrips[descrips.beer_id == 3366861]

,beer_id,beer_description
627,3366861,Limited Edition New England Style Double Hazy ...
135678,3366861,"After 23 years of coastal vibes, we’ve got ple..."


In [39]:
# just that one beer has two different descriptions.
descrips = descrips.drop_duplicates(subset=['beer_id'])
descrips.shape

(114347, 2)

In [40]:
descrips.to_csv('descriptions.csv', index=False)

In [41]:
bdf.columns

Index(['checkin_id', 'beer_id', 'user_id', 'rating_user', 'brewery_name',
       'beer_name', 'beer_style', 'brewery_id', 'brewery_type',
       'brewery_country', 'brewery_city', 'brewery_state', 'brewery_lat',
       'brewery_lon', 'venue_lat', 'venue_lon', 'venue_city', 'venue_country',
       'venue_state', 'venue_cat', 'venue_id', 'checkin_comment', 'venue_type',
       'rating_global', 'description', 'abv', 'date'],
      dtype='object')

In [43]:
bdf.drop('description', axis=1, inplace=True)

##### ============================================================================

In [46]:
df = pd.read_csv('bigframe.csv')
df.shape

(1425447, 27)

In [48]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [50]:
print([c for c in df.columns if c not in bdf.columns])
print([c for c in bdf.columns if c not in df.columns])

[]
[]


Make sure formats line up

Looks like I crossed up labels for 'venue_cat' and 'venue_type' for bdf, so need to switch them back.

In [72]:
bdf['venue_cat'] = bdf.venue_type
bdf['venue_type'] = ['[]' for _ in bdf.venue_type]  # stringy list, to match df

In [74]:
combo = pd.concat([df, bdf])

In [75]:
combo.shape

(2061965, 26)

In [76]:
combo.to_csv('comboframe.csv', index=False)